# Szeregowanie

In [ ]:
def scheduling():
    due_dates = [9, 8, 3, 3, 1, 5, 5, 1, 1]
    weights = [3, 26, 7, 22, 12, 11, 41, 40, 49]
    arr = [[i+1, due_dates[i], weights[i]] for i in range(len(due_dates))]
    arr.sort(key=lambda el: el[2], reverse=True)

    proc_timeline = [0 for _ in range(len(arr))]

    for i in range(len(arr)):
        el = arr[i]
        is_on_time = False

        arr_max = len(arr)-1
        if el[1] <= len(proc_timeline):
            arr_max = el[1]-1
        for j in range(arr_max, -1, -1):
            if proc_timeline[j] == 0:
                proc_timeline[j] = el[0]
                is_on_time = True
                break

        if not is_on_time:
            for j in range(len(arr)-1, -1, -1):
                if proc_timeline[j] == 0:
                    proc_timeline[j] = el[0]
                    break

    for el in range(len(arr)):
        print(f"Task: {proc_timeline[el]-1}, span from {el} to {el+1}")

scheduling()


Task: 8, span from 0 to 1
Task: 2, span from 1 to 2
Task: 3, span from 2 to 3
Task: 5, span from 3 to 4
Task: 6, span from 4 to 5
Task: 0, span from 5 to 6
Task: 4, span from 6 to 7
Task: 1, span from 7 to 8
Task: 7, span from 8 to 9


# Szeregowanie $L_{max}$ moze dziala


In [ ]:
def drozdo_is_not_your_friend():
    # Dane
    n = 4
    m = 3
    r = [0, 0, 1, 1]
    p = [2, 3, 4, 2]
    d = [1, 2, 3, 1]
    Lmax1 = []

    # Wypisywanie
    print("Podzielne zadania n = " + str(n)
          + "\nIdentyczne procesy m = " + str(m)
          + "\nMomenty gotowości r = " + str(r)
          + "\nCzas przetwarzania p = " + str(p)
          + "\nLinie krytyczne d = " + str(d))

    # Obliczamy wartość Lmax >= max{ri+pi-di, dla i w przedziale <1, n>}
    for i in range(n):
        Lmax1.append(r[i] + p[i] - d[i])
    Lmax2 = max(Lmax1)

    # Wypisywanie
    print("\n1.\n")
    print(f"Lmax >= max({Lmax1}) => Lmax >= {Lmax2}")

    # Obliczamy przedziały do których należy Lmax,
    # rx = {ri - d^T j, dla i oraz j w przedziale <1, n>}
    # d^T - d transponowane
    rx = []
    rall = []
    res = []

    for i in range(n):
        ri = []
        for j in range(n):
            ri.append(r[i]-d[j])
            rall.append(r[i]-d[j])
        rx.append(ri)

    # Każdy unikalny element należący do nowej macierzy rx
    # powstałej po działaniu ri - d^T j
    # Jest częścią przedziału prawostronnie domkniętego
    # tj. dla przykładu, jeśli do rx należy -5 i -4
    # to przedziały to (-inf; -5>, (-5;-4>, (-4; +inf)
    # na końcu dodajemy nowy przedział (Lmax2; +inf)
    # i robimy koniunkcję
    for i in rall:
        if i not in res:
            res.append(i)
    res.sort()
    res.append(Lmax2)
    final_interval = max(res)

    # Wypisywanie
    print("\n\n2.\n")
    for i in range(n):
        print(f"r {i+1} - d^T = {rx[i]}")
    print()

    print(f"Lmax in (-inf; {res[0]}>,", end=' ')
    for i in range(len(res)-1):
        print(f"({res[i]}; {res[i+1]},", end=' ')
    print(f"({res[-1]}; inf)")
    print(f"Lmax in <{Lmax2}; inf)")
    print(f"1. and 2. <=> Lmax in <{final_interval}; inf)")


    # Tworzymy nową linię krytyczną poprzez dodanie
    # do elementów starej początka przedziału
    # d' = d + test
    # tworzymy też wektor e, będący sumą mnogościową d' i r
    # jeżeli d = [0, 1, 2] i r = [1, 3, 4]
    # to e = [0, 1, 3, 4]
    test = final_interval + 2
    dtest = []
    for i in range(n):
        dtest.append(d[i] + test)
    e = []

    for i in range(n):
        e.append(r[i])
        e.append(dtest[i])
    e = list(set(e))

    # Wypisywanie
    print("\n\n3.\n")
    print(f"Wartość testowa: {test}")
    print(f"Testowa linia krytyczna d' = {dtest}")
    print(f"e = {e}")


    # Tworzymy wstępną sieć przepływową
    # Wierzchołki to: żródło s, zadania T1 ... Tn,
    # przedziały chwil czasowych [eji, ei+1] i ujście t
    # każde zadanie Ti wykonywać się może w przedziale <r[i], d'[i]>
    # łączymy więc każdy wierzchołek Ti z wszystkimi chwilami czasu
    # znajdującymi się w przedziale
    # etykieta łuku to szerokość przedziału ei+1 - ei
    # etykiety łuków od przedziałów do źródła wynikają ze wzoru m*(ei+1 - ei)
    arcs = []
    for i in range(n):
        arcs.append(("s", p[i], i + 1))

    for i in range(n):
        for j in range(len(e) - 1):
            if (r[i] <= e[j]) and (dtest[i] >= e[j+1]):
                arcs.append((i + 1, e[j + 1] - e[j], (e[j], e[j + 1])))

    for i in range(len(e) - 1):
        arcs.append(((e[i], e[i + 1]), (e[i + 1]-e[i]) * m, "t"))

    # Wypisywanie
    print("\n\n4.\n")
    print("s:")
    for i in range(n):
        print(f"\ts -->({arcs[i][1]})--> T{arcs[i][2]}")
    print()

    cur = 0
    for i in range(n, len(arcs) - len(e) + 1):
        if arcs[i][0] != cur:
            print(f"T{arcs[i][0]}")
            cur = arcs[i][0]
        print(f"\tT{arcs[i][0]} -->({arcs[i][1]})--> {arcs[i][2]}")
    print()

    print("Przedziały czasu:")
    for i in range(len(arcs) - len(e) + 1, len(arcs)):
        print(f"\t{arcs[i][0]} -->({arcs[i][1]})--> {arcs[i][2]}")

    # Przypisujemy zadania do wykonania na poszczególnych procesorach
    # Nasycamy wszystkie ścieżki, począwszy od góry
    # tj. przydzielamy czas do zadań mając na uwadze ograniczenia
    # wybieramy najmniejszy spośród:
    # pojemność procesorów w danym przedziale czasu (capacity)
    # długość przedziału czasu (lcap)
    # potrzebny nadal czas przetwarzania
    # jeżeli któreś zadanie nie zostanie wykonane, należy zwiększyć zmienną test
    capacity = {(e[i], e[i+1]): m*(e[i+1] - e[i]) for i in range(len(e) - 1)}
    needs = p.copy()
    new_arcs = []
    cur = 0

    for i in range(n, len(arcs) - len(e) + 1):
        if arcs[i][0] != cur:
            cur = arcs[i][0]
        lcap = arcs[i][1]
        used = min(capacity[arcs[i][2]], lcap, needs[cur - 1])
        needs[cur - 1] -= used
        capacity[arcs[i][2]] -= used
        new_arcs.append((cur, used, arcs[i][2]))

    # Wypisywanie
    print("\n\n5.\n")
    print("s:")
    for i in range(n):
        print(f"\ts -->({arcs[i][1]})--> T{arcs[i][2]}")
    print()

    cur = 0
    for i in range(len(new_arcs)):
        if arcs[i][0] != cur:
            print(f"T{new_arcs[i][0]}")
            cur = new_arcs[i][0]
        print(f"\tT{new_arcs[i][0]} -->({new_arcs[i][1]})--> {new_arcs[i][2]}")
    print()

    print("Przedziały czasu:")
    for i in range(len(arcs) - len(e) + 1, len(arcs)):
        print(f"\t{arcs[i][0]} -->({arcs[i][1]})--> {arcs[i][2]}")

    print("\nPozostałe zapotrzebowanie:")
    for i in range(len(needs)):
        print(f"\tT{i + 1}: {needs[i]}")

    # Sporządzamy teraz tabelę poszeregowania procesów
    # Dla każdego przedziału, dla każdego procesora
    # wybieramy proces do realizacji (można nie wybrać żadnego)
    # Pamiętamy, że nie można rozdzielać procesów na różne procesory
    # w trakcie jednego przedziału, tj. jeśli procesowi Ti przeznaczyliśmy
    # 2s w przedziale <x, y>, to może on być wykonywany tylko przez procesor Pj
#     if sum(needs) == 0:
#         proc_matrix = []
#         for i in range(m):
#             time = []
#             for j in range(len(e) - 1):
#                 time.extend([0] * (e[j + 1] - e[j]))
#             proc_matrix.append(time)

#         for i in range(len(new_arcs)):
#             possible = [j for j in range(new_arcs[i][2][0], new_arcs[i][2][1])]
#             need = new_arcs[i][1]
#             for j in range(need - 1):
#                 possible.pop()
#             placed = False
#             start_i = len(proc_matrix) - 1
#             while not placed:
#                 for pos in possible:
#                     if proc_matrix[start_i][pos] == 0:
#                         placed = True
#                         break
#                     while placed and need > 0:
#                         proc_matrix[start_i][pos] = new_arcs[i][0]
#                         pos += 1
#                         need -= 1
#                     start_i -= 1
    # ... [previous code remains unchanged]

# Revised section for assigning tasks to processors
    if sum(needs) == 0:
        # Initialize the processor matrix with 0 (indicates time slots where no task is assigned)
        proc_matrix = [[0] * e[-1] for _ in range(m)]

        # Iterate through each arc (task assignment) and try to place the task in the processor matrix
        for task_id, duration, time_interval in new_arcs:
            start_time, end_time = time_interval
            placed = False

            # Iterate through possible start times for the task within its time interval
            for start in range(start_time, end_time - duration + 1):
                # Check each processor to find a suitable one
                for proc_id, processor in enumerate(proc_matrix):
                    # Check if the processor can accommodate the task in the required time slots
                    if all(processor[i] == 0 for i in range(start, start + duration)):
                        # Assign the task to the time slots on the processor
                        for i in range(start, start + duration):
                            processor[i] = task_id
                        placed = True
                        break  # Task has been placed, move to the next task
                if placed:
                    break  # Task has been placed, no need to check further start times

            if not placed:
                print(f"Nie można przydzielić zadania {task_id} w przedziale {start_time}-{end_time}")
                break

        # Printing the processor matrix






        # Wypisywanie
        print('\n\n6.\n')
        print("", end='\t')
        for i in range(e[-1]):
            print(f"[{i}, {i + 1}]", end="\t")
        print()
        for i in range(m):
            print(f"P{m - i}", end="\t")
            for j in range(e[-1]):
                if proc_matrix[i][j] == 0:
                    print("-", end="\t")
                else:
                    print(f"T{proc_matrix[i][j]}", end="\t")
            print()

drozdo_is_not_your_friend()

Podzielne zadania n = 4
Identyczne procesy m = 3
Momenty gotowości r = [0, 0, 1, 1]
Czas przetwarzania p = [2, 3, 4, 2]
Linie krytyczne d = [1, 2, 3, 1]

1.

Lmax >= max([1, 1, 2, 2]) => Lmax >= 2


2.

r 1 - d^T = [-1, -2, -3, -1]
r 2 - d^T = [-1, -2, -3, -1]
r 3 - d^T = [0, -1, -2, 0]
r 4 - d^T = [0, -1, -2, 0]

Lmax in (-inf; -3>, (-3; -2, (-2; -1, (-1; 0, (0; 2, (2; inf)
Lmax in <2; inf)
1. and 2. <=> Lmax in <2; inf)


3.

Wartość testowa: 4
Testowa linia krytyczna d' = [5, 6, 7, 5]
e = [0, 1, 5, 6, 7]


4.

s:
	s -->(2)--> T1
	s -->(3)--> T2
	s -->(4)--> T3
	s -->(2)--> T4

T1
	T1 -->(1)--> (0, 1)
	T1 -->(4)--> (1, 5)
T2
	T2 -->(1)--> (0, 1)
	T2 -->(4)--> (1, 5)
	T2 -->(1)--> (5, 6)
T3
	T3 -->(4)--> (1, 5)
	T3 -->(1)--> (5, 6)
	T3 -->(1)--> (6, 7)
T4
	T4 -->(4)--> (1, 5)

Przedziały czasu:
	(0, 1) -->(3)--> t
	(1, 5) -->(12)--> t
	(5, 6) -->(3)--> t
	(6, 7) -->(3)--> t


5.

s:
	s -->(2)--> T1
	s -->(3)--> T2
	s -->(4)--> T3
	s -->(2)--> T4

T1
	T1 -->(1)--> (0, 1)
T1
	T1 -->(1)-

# Huffman bezstratna kompresja i nie dziala

In [ ]:
def huffman():
    # Node of a Huffman Tree
    class Nodes:
        def __init__(self, probability, symbol, left = None, right = None):
            # probability of the symbol
            self.probability = probability

            # the symbol
            self.symbol = symbol

            # the left node
            self.left = left

            # the right node
            self.right = right

            # the tree direction (0 or 1)
            self.code = ''

    """ A supporting function in order to calculate the probabilities of symbols in specified data """
    def CalculateProbability(the_data):
        the_symbols = dict()
        for item in the_data:
            if the_symbols.get(item) == None:
                the_symbols[item] = 1
            else:
                the_symbols[item] += 1
        return the_symbols

    """ A supporting function in order to print the codes of symbols by travelling a Huffman Tree """
    the_codes = dict()

    def CalculateCodes(node, value = ''):
        # a huffman code for current node
        newValue = value + str(node.code)

        if(node.left):
            CalculateCodes(node.left, newValue)
        if(node.right):
            CalculateCodes(node.right, newValue)

        if(not node.left and not node.right):
            the_codes[node.symbol] = newValue

        return the_codes

    """ A supporting function in order to get the encoded result """
    def OutputEncoded(the_data, coding):
        encodingOutput = []
        for element in the_data:
            # print(coding[element], end = '')
            encodingOutput.append(coding[element])

        the_string = ''.join([str(item) for item in encodingOutput])
        return the_string

    """ A supporting function in order to calculate the space difference between compressed and non compressed data"""
    def TotalGain(the_data, coding):
        # total bit space to store the data before compression
        beforeCompression = len(the_data) * 8
        afterCompression = 0
        the_symbols = coding.keys()
        for symbol in the_symbols:
            the_count = the_data.count(symbol)
            # calculating how many bit is required for that symbol in total
            afterCompression += the_count * len(coding[symbol])
        print("Space usage before compression (in bits):", beforeCompression)
        print("Space usage after compression (in bits):",  afterCompression)

    def HuffmanEncoding(the_data):
        symbolWithProbs = CalculateProbability(the_data)
        the_symbols = symbolWithProbs.keys()
        the_probabilities = symbolWithProbs.values()
        print("symbols: ", the_symbols)
        print("probabilities: ", the_probabilities)

        the_nodes = []

        # converting symbols and probabilities into huffman tree nodes
        for symbol in the_symbols:
            the_nodes.append(Nodes(symbolWithProbs.get(symbol), symbol))

        while len(the_nodes) > 1:
            # sorting all the nodes in ascending order based on their probability
            the_nodes = sorted(the_nodes, key = lambda x: x.probability)
            # for node in nodes:
            #      print(node.symbol, node.prob)

            # picking two smallest nodes
            right = the_nodes[0]
            left = the_nodes[1]

            left.code = 0
            right.code = 1

            # combining the 2 smallest nodes to create new node
            newNode = Nodes(left.probability + right.probability, left.symbol + right.symbol, left, right)

            the_nodes.remove(left)
            the_nodes.remove(right)
            the_nodes.append(newNode)

        huffmanEncoding = CalculateCodes(the_nodes[0])
        print("symbols with codes", huffmanEncoding)
        TotalGain(the_data, huffmanEncoding)
        encodedOutput = OutputEncoded(the_data,huffmanEncoding)
        return encodedOutput, the_nodes[0]

    def HuffmanDecoding(encodedData, huffmanTree):
        treeHead = huffmanTree
        decodedOutput = []
        for x in encodedData:
            if x == '1':
                huffmanTree = huffmanTree.right
            elif x == '0':
                huffmanTree = huffmanTree.left
            try:
                if huffmanTree.left.symbol == None and huffmanTree.right.symbol == None:
                    pass
            except AttributeError:
                decodedOutput.append(huffmanTree.symbol)
                huffmanTree = treeHead

        string = ''.join([str(item) for item in decodedOutput])
        return string


    the_data = "SEQUENCE"


    print(the_data)
    encoding, the_tree = HuffmanEncoding(the_data)
    print("Encoded output", encoding)
    print("Decoded Output", HuffmanDecoding(encoding, the_tree))

huffman()

SEQUENCE
symbols:  dict_keys(['S', 'E', 'Q', 'U', 'N', 'C'])
probabilities:  dict_values([1, 3, 1, 1, 1, 1])
symbols with codes {'E': '00', 'N': '010', 'U': '011', 'Q': '100', 'S': '101', 'C': '11'}
Space usage before compression (in bits): 64
Space usage after compression (in bits): 20
Encoded output 10100100011000101100
Decoded Output SEQUENCE


# McNaughton Poprawiony szeregowanie z wywlaszczaniem p|pmtn|cmax

In [ ]:
import statistics as st


def naughty_man():
    # how many machines (P)
    #TU ZMIENIAJ
    machines_size = 3 # P machines
    machine_time_left = [0 for _ in range(machines_size)]
    machine_timelines = [[] for _ in range(machines_size)]
    # proc. times
    #TU ZMIENIAJ
    proc_times = [1, 2, 4, 3, 5, 6, 2]# czasy procesow
    cmax = max(sum(proc_times)/machines_size, max(proc_times))
    # print(cmax)

    arr = [[i + 1, proc_times[i]] for i in range(len(proc_times))]

    current_machine = 0
    for proc in arr:
        if machine_time_left[current_machine] + proc[1] <= cmax:
            machine_time_left[current_machine] += proc[1]
            machine_timelines[current_machine].append(proc)
        else:
            time_left = cmax - machine_time_left[current_machine]
            time_new = proc[1] - time_left
            machine_time_left[current_machine] = cmax
            machine_timelines[current_machine].append([proc[0], time_left])
            current_machine += 1
            machine_timelines[current_machine].append([proc[0], time_new])
            machine_time_left[current_machine] += time_new

    for i in range(machines_size):
        print(f"Machine {i + 1} with jobs timeline (number, timespan): {machine_timelines[i]}")


naughty_man()

Machine 1 with jobs timeline (number, timespan): [[1, 1], [2, 2], [3, 4], [4, 0.666666666666667]]
Machine 2 with jobs timeline (number, timespan): [[4, 2.333333333333333], [5, 5], [6, 0.3333333333333339]]
Machine 3 with jobs timeline (number, timespan): [[6, 5.666666666666666], [7, 2]]


# Knapsack 0-1

In [ ]:
def binary_knapsack():
     # !!! 0 na poczatku !!!
    w = [0, 8, 1, 2, 9, 9, 9, 2] # wartosci
    s = [0, 8, 8, 6, 8, 7, 3, 8] # rozmiary
    show_bigger_values = False #TU MUSI BYC FALSE
    # show_bigger_values - czy wyświetlać wartości większe niż pojemność
    # to execute properly, leave first element
    # of each list as 0

    max_value = sum(w)
    capacity = 11 #TU TEZ TRZEBA ZMIENIC

    INF = 100
    # Wypisanie tablicy
    def print_table(T):
        for i in range(1, len(T)):
            print(i, end='\t')
            for j in range(1, len(T[0])):
                if T[i][j] == INF:
                    print("-", end="\t")
                else:
                    print(T[i][j], end="\t")
            print()
        with open("knapsack.txt", 'w') as f:
            f.write("\t")
            for i in range(1, len(T[0])):
                f.write(f"{i}\t")
            f.write("\n")
            for i in range(1, len(T)):
                f.write(f"{i}\t")
                for j in range(1, len(T[0])):
                    if T[i][j] == INF:
                        f.write("-\t")
                    else:
                        f.write(f"{T[i][j]}\t")
                f.write("\n")

    # Znalezienie rozwiązania
    def find_solution(T, s):
        start_i = len(w) - 1
        start_j = sum(w)
        used = []
        for j in range(start_j, 0, -1):
            if T[-1][j] != INF:
                start_j = j
                break
        val = T[start_i][start_j]
        solution = start_j
        while True:
            while True:
                if T[start_i - 1][start_j] != T[start_i][start_j]:
                    break
                start_i -= 1
            used.append(start_i)
            val -= s[start_i]
            if val == 0:
                break
            start_i -= 1
            for j in range(start_j, 0, -1):
                if T[start_i][j] == val:
                    start_j = j
                    break

        print(f"\n\nWartość plecaka = {solution}\nUżyte elementy: ", end="")
        print(*sorted(used), sep=", ")

    # number of columns to calculate
    g = [[100]*(max_value + 1) for i in range(len(w))]

    for i in range(len(g)):
        g[i][0] = 0

    for j in range(1, len(g[0])):
        g[0][j] = INF

    if(show_bigger_values):
        for i in range(1, len(g)):
            for j in range(1, len(g[i])):
                new_value = g[i-1][j - w[i]] + s[i]
                if w[i] <= j:
                    g[i][j] = min(new_value, g[i-1][j])
                else:
                    g[i][j] = g[i-1][j]
    else:
        for i in range(1, len(g)):
            for j in range(1, len(g[i])):
                new_value = g[i-1][j - w[i]] + s[i]
                if w[i] <= j and new_value <= capacity:
                    g[i][j] = min(new_value, g[i-1][j])
                else:
                    g[i][j] = g[i-1][j]

    header = [i for i in range(1, max_value + 1)]
    print("", end="\t")
    print(*header, sep="\t")
    print_table(g)
    find_solution(g, s)
binary_knapsack()




	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	33	34	35	36	37	38	39	40
1	-	-	-	-	-	-	-	8	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	
2	8	-	-	-	-	-	-	8	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	
3	8	6	-	-	-	-	-	8	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	
4	8	6	-	-	-	-	-	8	8	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	
5	8	6	-	-	-	-	-	8	7	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	
6	8	6	-	-	-	-	-	8	3	11	9	-	-	-	-	-	11	10	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	
7	8	6	-	-	-	-	-	8	3	11	9	-	-	-	-	-	11	10	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	


Wartość plecaka = 18
Użyte elementy: 5, 6
	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	33	34	35	36	37	38	39	40
1	-	-	-	-	-	-	-	8	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	
2	8	-	-	-	-	-	-	8	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	-	

# Knapsack 0-1 pretty print druga metoda

In [ ]:
def binary_knapsack_pretty():
    import pandas as pd
     # !!! 0 na poczatku !!!
    w = [0, 3, 4, 2, 6, 1] # wartosci
    s = [0, 5, 3, 2, 4, 3] # rozmiary
    show_bigger_values = False #TU ZAWSZE FALSE
    # show_bigger_values - czy wyświetlać wartości większe niż pojemność
    # to execute properly, leave first element
    # of each list as 0

    max_value = sum(w)
    capacity = 10 #TU ZMIEN

    INF = 100
    # Wypisanie tablicy
    def print_table(T):
        # change all INF to -
        data = [[i if i != INF else "" for i in row] for row in T]
        pd.set_option('display.max_columns', None)
        df = pd.DataFrame(data)
        display(df)

    # Znalezienie rozwiązania
    def find_solution(T, s):
        start_i = len(w) - 1
        start_j = sum(w)
        used = []
        for j in range(start_j, 0, -1):
            if T[-1][j] != INF:
                start_j = j
                break
        val = T[start_i][start_j]
        solution = start_j
        while True:
            while True:
                if T[start_i - 1][start_j] != T[start_i][start_j]:
                    break
                start_i -= 1
            used.append(start_i)
            val -= s[start_i]
            if val == 0:
                break
            start_i -= 1
            for j in range(start_j, 0, -1):
                if T[start_i][j] == val:
                    start_j = j
                    break

        print(f"\n\nWartość plecaka = {solution}\nUżyte elementy: ", end="")
        print(*sorted(used), sep=", ")

    # number of columns to calculate
    g = [[100]*(max_value + 1) for i in range(len(w))]

    for i in range(len(g)):
        g[i][0] = 0

    for j in range(1, len(g[0])):
        g[0][j] = INF

    if(show_bigger_values):
        for i in range(1, len(g)):
            for j in range(1, len(g[i])):
                new_value = g[i-1][j - w[i]] + s[i]
                if w[i] <= j:
                    g[i][j] = min(new_value, g[i-1][j])
                else:
                    g[i][j] = g[i-1][j]
    else:
        for i in range(1, len(g)):
            for j in range(1, len(g[i])):
                new_value = g[i-1][j - w[i]] + s[i]
                if w[i] <= j and new_value <= capacity:
                    g[i][j] = min(new_value, g[i-1][j])
                else:
                    g[i][j] = g[i-1][j]

    print_table(g)
    find_solution(g, s)

binary_knapsack_pretty()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0,,,,,,,,,,,,,,,,
1,0,,,5,,,,,,,,,,,,,
2,0,,,5,3,,,8,,,,,,,,,
3,0,,2,5,3,7,5,8,,10,,,,,,,
4,0,,2,5,3,7,4,8,6,9,7,11,9,12,,14,
5,0,3,2,5,3,6,4,7,6,9,7,10,9,12,15,14,17




Wartość plecaka = 16
Użyte elementy: 1, 2, 3, 4, 5


# Knapsack ciagly pierwsza metoda


In [ ]:
def real_knapsack():
    # knapsack size
    b = 11

    w = [8, 1, 2, 9, 9, 9, 2] # wartosci
    s = [8, 8, 6, 8, 7, 3, 8] # rozmiary

    items = [[s[i], w[i]] for i in range(len(w))]


    for i in range(len(items)):
        items[i].append(i+1)

    items = sorted(items, key=lambda item: item[1]/item[0], reverse=True)

    print("Reordered items: ")
    for item in items:
        print("ID: {}, SIZE: {}, VALUE: {}, Wi/Si: {}".format(item[2], item[0], item[1], item[1]/item[0]))

    knapsack = []
    i = 0
    while b - items[i][0] >= 0:
        b -= items[i][0]
        knapsack.append(items[i])
        i += 1

    if b != 0:
        items[i][1] = items[i][1] * b / items[i][0]
        items[i][0] = b
        knapsack.append(items[i])

    print("Knapsack value: {}".format(sum(x[1] for x in knapsack)))
    print("Items in knapsack: ")
    for item in knapsack:
        print("ID: {}, SIZE: {}, VALUE: {}".format(item[2], item[0], item[1]))


real_knapsack()



# Critical path; ready, due times; (CPM)

In [ ]:
def critical_path():
    from collections import defaultdict
    def checkProcedessors(executionTime, readyTime, startingNode, Node):
        for proc in executionTime:
            for suc in executionTime[proc]:
                if suc == Node:
                    if readyTime[proc] == 0 and proc != startingNode:
                        return False
        return True

    def checkSuccesors(executionTime, dueTime, Node):
        for proc in executionTime:
            for suc in executionTime[proc]:
                if proc == Node:
                    if dueTime[suc] == 0:
                        return False
        return True

    def getMaxLenght(executionTime, readyTime, Node):
        maxLen = 0
        for proc in executionTime:
            for suc in executionTime[proc]:
                if suc == Node:
                    if readyTime[proc] + executionTime[proc][suc] > maxLen:
                        maxLen = readyTime[proc] + executionTime[proc][suc]
        return maxLen

    def getMinLength(executionTime, dueTime, Node):
        minLen = float("inf")
        for proc in executionTime:
            for suc in executionTime[proc]:
                if proc == Node:
                    if dueTime[suc] - executionTime[proc][suc] < minLen:
                        minLen = dueTime[suc] - executionTime[proc][suc]
        return minLen

    def countReadyTimes(executionTime, readyTime, startingNode, endingNode):
        while readyTime[endingNode] == 0:
            for Node in range(startingNode + 1, endingNode + 1):
                if readyTime[Node] == 0 and checkProcedessors(executionTime,
                                                              readyTime,
                                                              startingNode,
                                                              Node):
                    readyTime[Node] = getMaxLenght(executionTime, readyTime, Node)

    def countDueTimes(executionTime, readyTime, dueTime, startingNode, endingNode):
        dueTime[endingNode] = readyTime[endingNode]
        while True:
            for Node in range(endingNode - 1, startingNode - 1, -1):
                if dueTime[Node] == 0 and checkSuccesors(executionTime,
                                                         dueTime,
                                                         Node):
                    dueTime[Node] = getMinLength(executionTime, dueTime, Node)
                    if Node == 0:
                        return

    def Dfs(executionTime, readyTime, dueTime, currentNode,
            endingNode, cirticalPaths, path):
        if currentNode == endingNode:
            cirticalPaths.append(path)
            return

        for proc in executionTime:
            for suc in executionTime[proc]:
                if proc == currentNode:
                    tn = readyTime[currentNode] + executionTime[proc][suc]
                    if readyTime[suc] == dueTime[suc] and tn == readyTime[suc]:
                        nextStepPath = path.copy()
                        nextStepPath.append(suc)
                        Dfs(executionTime, readyTime, dueTime, suc,
                            endingNode, cirticalPaths, nextStepPath)

    def findCriticalPaths(executionTime, readyTime, dueTime,
                          startingNode, endingNode):
        cirticalPaths = []
        Dfs(executionTime, readyTime, dueTime, startingNode,
            endingNode, cirticalPaths, [startingNode])
        return cirticalPaths
    # startingNode zawsze musi być o najniższym
    # indeksie, a endingNode najwyższym!!!!
    # indeksy muszą iść po kolei!!!
    verticesNumber = 9 #ilosc wierzcholkow
    executionTime = defaultdict(dict)
    startingNode = 0 #najnizszy
    endingNode = verticesNumber - 1 #najwyzszy
    #wstawiasz po kolei
    executionTime[0][1] = 1
    executionTime[0][2] = 5
    executionTime[0][3] = 6
    executionTime[0][5] = 5
    executionTime[0][6] = 5
    executionTime[1][2] = 1
    executionTime[1][5] = 2
    executionTime[1][6] = 10
    executionTime[1][7] = 7
    executionTime[2][3] = 1
    executionTime[2][4] = 10
    executionTime[2][5] = 5
    executionTime[3][4] = 1
    executionTime[4][5] = 1
    executionTime[5][6] = 1
    executionTime[6][7] = 1
    executionTime[6][8] = 2
    executionTime[7][8] = 1


    readyTime = [0] * verticesNumber
    dueTime = [0] * verticesNumber
    countReadyTimes(executionTime, readyTime, startingNode, endingNode)
    countDueTimes(executionTime, readyTime, dueTime, startingNode, endingNode)
    print("ready times:\t", readyTime)
    print("due times:\t", dueTime)
    print("critical paths:\t", findCriticalPaths(executionTime, readyTime,dueTime, startingNode, endingNode))

critical_path()

ready times:	 [0, 1, 5, 6, 15, 16, 17, 18, 19]
due times:	 [0, 4, 5, 14, 15, 16, 17, 18, 19]
critical paths:	 [[0, 2, 4, 5, 6, 7, 8], [0, 2, 4, 5, 6, 8]]


# Floyd

In [ ]:
def floyd():
    I = 100 # Nieskonczonosc w zaleznosci od polecenia

    initMatrix = [[0, I, 7, I, 5, I],
                  [I, 0, I, I, 8, 4],
                  [I, I, 0, 5, I, 9],
                  [2, I, I, 0, I, 3],
                  [9, I, 9, I, 0, I],
                  [I, 1, 7, I, I, 0]]

    def print_res(nIter, numberOfVertices):
        for iterNumber in range(0, numberOfVertices):
            print("iterNumber=", iterNumber+1)

            for i in range(0, numberOfVertices):
                for j in range(0, numberOfVertices):
                    print(nIter[iterNumber][i][j], end='\t')
                print()
            print()


    numberOfVertices = len(initMatrix)
    nIter = []

    for iterNumber in range(0, numberOfVertices):
        if iterNumber == 0:
            nIter.append(list(map(list, initMatrix)))
        else:
            nIter.append(list(map(list, nIter[iterNumber - 1])))

        for i in range(0, numberOfVertices):
            for j in range(0, numberOfVertices):
                if i == iterNumber or j == iterNumber:
                    continue

                nIter[iterNumber][i][j] = min(
                    nIter[iterNumber][i][j],
                    nIter[iterNumber][i][iterNumber]
                    + nIter[iterNumber][iterNumber][j])


    print("Initial Matrix:")
    for i in range(0, numberOfVertices):
        for j in range(0, numberOfVertices):
            print(initMatrix[i][j], end='\t')
        print()
    print()
    print_res(nIter, numberOfVertices)

floyd()

Initial Matrix:
0	100	7	100	5	100	
100	0	100	100	8	4	
100	100	0	5	100	9	
2	100	100	0	100	3	
9	100	9	100	0	100	
100	1	7	100	100	0	

iterNumber= 1
0	100	7	100	5	100	
100	0	100	100	8	4	
100	100	0	5	100	9	
2	100	9	0	7	3	
9	100	9	100	0	100	
100	1	7	100	100	0	

iterNumber= 2
0	100	7	100	5	100	
100	0	100	100	8	4	
100	100	0	5	100	9	
2	100	9	0	7	3	
9	100	9	100	0	100	
100	1	7	100	9	0	

iterNumber= 3
0	100	7	12	5	16	
100	0	100	100	8	4	
100	100	0	5	100	9	
2	100	9	0	7	3	
9	100	9	14	0	18	
100	1	7	12	9	0	

iterNumber= 4
0	100	7	12	5	15	
100	0	100	100	8	4	
7	100	0	5	12	8	
2	100	9	0	7	3	
9	100	9	14	0	17	
14	1	7	12	9	0	

iterNumber= 5
0	100	7	12	5	15	
17	0	17	22	8	4	
7	100	0	5	12	8	
2	100	9	0	7	3	
9	100	9	14	0	17	
14	1	7	12	9	0	

iterNumber= 6
0	16	7	12	5	15	
17	0	11	16	8	4	
7	9	0	5	12	8	
2	4	9	0	7	3	
9	18	9	14	0	17	
14	1	7	12	9	0	



# Floyd pretty print

In [ ]:
def floyd_pretty():
    import pandas as pd
    I = 100 # Nieskonczonosc w zaleznosci od polecenia
    #to macierz grafu
    initMatrix = [[0, 3, I, I, I],
                  [I, 0, -2, -3, -2],
                  [1, I, 0, 2, I],
                  [I, I, I, 0, I],
                  [I, I, I, 1, 0]]

    def print_res(nIter, numberOfVertices):
        for iterNumber in range(0, numberOfVertices):
            print("iterNumber=", iterNumber+1)

            pd.set_option('display.max_columns', None)
            df = pd.DataFrame(nIter[iterNumber])
            display(df)


    numberOfVertices = len(initMatrix)
    nIter = []

    for iterNumber in range(0, numberOfVertices):
        if iterNumber == 0:
            nIter.append(list(map(list, initMatrix)))
        else:
            nIter.append(list(map(list, nIter[iterNumber - 1])))

        for i in range(0, numberOfVertices):
            for j in range(0, numberOfVertices):
                if i == iterNumber or j == iterNumber:
                    continue

                nIter[iterNumber][i][j] = min(
                    nIter[iterNumber][i][j],
                    nIter[iterNumber][i][iterNumber]
                    + nIter[iterNumber][iterNumber][j])


    print("Initial Matrix:")
    df = pd.DataFrame(initMatrix)
    display(df)

    print_res(nIter, numberOfVertices)

floyd_pretty()

Initial Matrix:


,0,1,2,3,4
0,0,3,100,100,100
1,100,0,-2,-3,-2
2,1,100,0,2,100
3,100,100,100,0,100
4,100,100,100,1,0


iterNumber= 1


,0,1,2,3,4
0,0,3,100,100,100
1,100,0,-2,-3,-2
2,1,4,0,2,100
3,100,100,100,0,100
4,100,100,100,1,0


iterNumber= 2


...

,0,1,2,3,4
0,0,3,1,0,1
1,-1,0,-2,-3,-2
2,1,4,0,1,2
3,99,100,98,0,98
4,99,100,98,1,0


iterNumber= 4


,0,1,2,3,4
0,0,3,1,0,1
1,-1,0,-2,-3,-2
2,1,4,0,1,2
3,99,100,98,0,98
4,99,100,98,1,0


iterNumber= 5


,0,1,2,3,4
0,0,3,1,0,1
1,-1,0,-2,-3,-2
2,1,4,0,1,2
3,99,100,98,0,98
4,99,100,98,1,0




# Dinitz max flow directed graph with capacity

In [ ]:
def dinitz():
    from networkx.algorithms.flow import dinitz
    import networkx as nx

    G = nx.DiGraph()
    #tutaj dodajesz krawedzie etykieta, cel, flow, capacity
    G.add_edge("S", 2, flow=1, capacity=8)
    G.add_edge("S", 3, flow=0, capacity=9)
    G.add_edge("S", 5, flow=1, capacity=5)
    G.add_edge(2, 4, flow=1, capacity=3)
    G.add_edge(2, 5, flow=3, capacity=3)
    G.add_edge(3, 5, flow=0, capacity=5)
    G.add_edge(4, "T", flow=3, capacity=9)
    G.add_edge(5, 4, flow=2, capacity=7)
    G.add_edge(5, "T", flow=2, capacity=8)



    R = dinitz(G, "S", "T")
    flow_value, flow_dict = nx.maximum_flow(R, "S", "T")
    print("Flow Value: ", flow_value)
    display(flow_dict)

dinitz()

Flow Value:  16


{'S': {2: 6, 3: 5, 5: 5},
 2: {'S': 0, 4: 3, 5: 3},
 3: {'S': 0, 5: 5},
 5: {'S': 0, 2: 0, 3: 0, 4: 5, 'T': 8},
 4: {2: 0, 5: 0, 'T': 8},
 'T': {5: 0, 4: 0}}

# MIN CUT dinitz znalezienie waskiego gardla battleneck

In [ ]:
def min_cut():
    from networkx.algorithms.flow import dinitz
    import networkx as nx

    G = nx.DiGraph()
    G.add_edge("S", 2, flow=1, capacity=8)
    G.add_edge("S", 3, flow=0, capacity=9)
    G.add_edge("S", 5, flow=1, capacity=5)
    G.add_edge(2, 4, flow=1, capacity=3)
    G.add_edge(2, 5, flow=3, capacity=3)
    G.add_edge(3, 5, flow=0, capacity=5)
    G.add_edge(4, "T", flow=3, capacity=9)
    G.add_edge(5, 4, flow=2, capacity=7)
    G.add_edge(5, "T", flow=2, capacity=8)

    R = dinitz(G, "S", "T")
    cut_value, partition = nx.minimum_cut(R, "S", "T")
    print("Cut Value: ", cut_value)
    display(partition)

min_cut()

Cut Value:  16


({2, 3, 'S'}, {4, 5, 'T'})

#Kruskal minimum spanning tree of an undirected edge-weighted graph drzewo rospinajace minimalne

In [ ]:
# Python program for Kruskal's algorithm to find
# Minimum Spanning Tree of a given connected,
# undirected and weighted graph

# Class to represent a graph

def kruskal():


    # Driver's code
    # A - 0
    # B - 1
    # C - 2
    # D - 3
    # E - 4
    # F - 5
    # G - 6

    # Liczba wierzchołków
    liczba_wierzcholkow = 7
    class Graph:

        def __init__(self, vertices):
            self.V = vertices  # No. of vertices
            self.graph = []
            # to store graph

        # function to add an edge to graph
        def addEdge(self, u, v, w):
            self.graph.append([u, v, w])

        # A utility function to find set of an element i
        # (truly uses path compression technique)
        def find(self, parent, i):
            if parent[i] != i:
              # Reassignment of node's parent to root node as
              # path compression requires
                parent[i] = self.find(parent, parent[i])
            return parent[i]

        # A function that does union of two sets of x and y
        # (uses union by rank)
        def union(self, parent, rank, x, y):

            # Attach smaller rank tree under root of
            # high rank tree (Union by Rank)
            if rank[x] < rank[y]:
                parent[x] = y
            elif rank[x] > rank[y]:
                parent[y] = x

            # If ranks are same, then make one as root
            # and increment its rank by one
            else:
                parent[y] = x
                rank[x] += 1

        # The main function to construct MST using Kruskal's
            # algorithm
        def KruskalMST(self):

            result = []  # This will store the resultant MST

            # An index variable, used for sorted edges
            i = 0

            # An index variable, used for result[]
            e = 0

            # Step 1:  Sort all the edges in
            # non-decreasing order of their
            # weight.  If we are not allowed to change the
            # given graph, we can create a copy of graph
            self.graph = sorted(self.graph,
                                key=lambda item: item[2])

            parent = []
            rank = []

            # Create V subsets with single elements
            for node in range(self.V):
                parent.append(node)
                rank.append(0)

            # Number of edges to be taken is less than to V-1
            while e < self.V - 1:

                # Step 2: Pick the smallest edge and increment
                # the index for next iteration
                u, v, w = self.graph[i]
                i = i + 1
                x = self.find(parent, u)
                y = self.find(parent, v)

                # If including this edge doesn't
                # cause cycle, then include it in result
                # and increment the index of result
                # for next edge
                if x != y:
                    e = e + 1
                    result.append([u, v, w])
                    self.union(parent, rank, x, y)
                # Else discard the edge

            minimumCost = 0
            print("Edges in the constructed MST")
            for u, v, weight in result:
                minimumCost += weight
                print("%d -- %d == %d" % (u, v, weight))
            print("Minimum Spanning Tree", minimumCost)
    #u->v z weight
    g = Graph(liczba_wierzcholkow)
    g.addEdge(0, 1, 7)
    g.addEdge(0, 3, 5)
    g.addEdge(1, 2, 8)
    g.addEdge(1, 3, 9)
    g.addEdge(1, 4, 7)
    g.addEdge(2, 4, 5)
    g.addEdge(3, 4, 15)
    g.addEdge(3, 5, 6)
    g.addEdge(4, 5, 8)
    g.addEdge(4, 6, 9)
    g.addEdge(5, 6, 11)
    # Function call
    g.KruskalMST()

    # This code is contributed by Neelam Yadav
    # Improved by James Graça-Jones
kruskal()

Edges in the constructed MST
0 -- 3 == 5
2 -- 4 == 5
3 -- 5 == 6
0 -- 1 == 7
1 -- 4 == 7
4 -- 6 == 9
Minimum Spanning Tree 39


# Kruskal V2

In [ ]:
def kruskal_v2():
    from networkx.algorithms.flow import dinitz
    import networkx as nx

    g = nx.Graph()
    g.add_edge(0, 1, weight=7)
    g.add_edge(0, 3, weight=5)
    g.add_edge(1, 2, weight=8)
    g.add_edge(1, 3, weight=9)
    g.add_edge(1, 4, weight=7)
    g.add_edge(2, 4, weight=5)
    g.add_edge(3, 4, weight=15)
    g.add_edge(3, 5, weight=6)
    g.add_edge(4, 5, weight=8)
    g.add_edge(4, 6, weight=9)
    g.add_edge(5, 6, weight=11)

    T = nx.minimum_spanning_tree(g)
    print("Minimum Spanning Tree", T)
    print("Weight sum: ", sum([T.get_edge_data(u, v).get("weight", 0) for u, v in T.edges()]))
    for u, v in T.edges():
        weight = T.get_edge_data(u, v).get("weight", 0)
        print(f"{u} -- {v} == {weight}")
kruskal_v2()

Minimum Spanning Tree Graph with 7 nodes and 6 edges
Weight sum:  39
0 -- 3 == 5
0 -- 1 == 7
1 -- 4 == 7
3 -- 5 == 6
2 -- 4 == 5
4 -- 6 == 9


# Bellman-Ford  computes shortest paths from a single source vertex to all of the other vertices in a weighted digraph.

In [ ]:
def bellman_ford():
    import networkx as nx

    g = nx.DiGraph()
    g.add_edge(0, 1, weight=7)
    g.add_edge(0, 3, weight=5)
    g.add_edge(1, 2, weight=8)
    g.add_edge(1, 3, weight=9)
    g.add_edge(1, 4, weight=7)
    g.add_edge(2, 4, weight=5)
    g.add_edge(3, 4, weight=15)
    g.add_edge(3, 5, weight=6)
    g.add_edge(4, 5, weight=8)
    g.add_edge(4, 6, weight=9)
    g.add_edge(5, 6, weight=11)

    path = nx.bellman_ford_path(g, 0, 6)
    display(path)
    print("Weight sum: ", sum([g.get_edge_data(u, v).get("weight", 0) for u, v in zip(path, path[1:])]))
bellman_ford()

[0, 3, 5, 6]

Weight sum:  22


# Manually typed Bellman-Ford odleglosc od sourca
## (przez manually typed mam na myśli, że copilot go napisał XD)

In [ ]:
def bellman_ford_v2():
    import pandas as pd
    # Python program for Bellman-Ford's single source
    # shortest path algorithm.

    # Class to represent a graph
    class Graph:

        def __init__(self, vertices):
            self.V = vertices  # No. of vertices
            self.graph = []  # default dictionary to store graph

        # function to add an edge to graph
        def addEdge(self, u, v, w):
            self.graph.append([u, v, w])

        # utility function used to print the solution
        def printArr(self, dist):
            print("Vertex Distance from Source")
            df = pd.DataFrame(dist)
            display(df)

        # The main function that finds shortest distances from src to
        # all other vertices using Bellman-Ford algorithm. The function
        # also detects negative weight cycle
        def BellmanFord(self, src):

            # Step 1: Initialize distances from src to all other vertices
            # as INFINITE
            dist = [float("Inf")] * self.V
            dist[src] = 0

            # Step 2: Relax all edges |V| - 1 times. A simple shortest
            # path from src to any other vertex can have at-most |V| - 1
            # edges
            for _ in range(self.V - 1):
                # Update dist value and parent index of the adjacent vertices of
                # the picked vertex. Consider only those vertices which are still in
                # queue
                for u, v, w in self.graph:
                    if dist[u] != float("Inf") and dist[u] + w < dist[v]:
                        dist[v] = dist[u] + w

            # Step 3: check for negative-weight cycles. The above step
            # guarantees shortest distances if graph doesn't contain
            # negative weight cycle. If we get a shorter path, then there
            # is a cycle.
            for u, v, w in self.graph:
                if dist[u] != float("Inf") and dist[u] + w < dist[v]:
                    print("Graph contains negative weight cycle")
                    return

            # print all distance
            self.printArr(dist)

    g = Graph(7)
    g.addEdge(0, 1, 7)
    g.addEdge(0, 3, 5)
    g.addEdge(1, 2, 8)
    g.addEdge(1, 3, 9)
    g.addEdge(1, 4, 7)
    g.addEdge(2, 4, 5)
    g.addEdge(3, 4, 15)
    g.addEdge(3, 5, 6)
    g.addEdge(4, 5, 8)
    g.addEdge(4, 6, 9)
    g.addEdge(5, 6, 11)

    g.BellmanFord(0)

bellman_ford_v2()

Vertex Distance from Source


,0
0,0
1,7
2,15
3,5
4,14
5,11
6,22


# Prim's Algorithm a greedy algorithm that finds a minimum spanning tree for a weighted undirected graph

In [ ]:
def prim():
    INF = 9999999
    # number of vertices
    V = 6
    # graph adjacency
    G = [[0, 2, 3, 3, 0, 0],
         [2, 0, 4, 0, 3, 0],
         [3, 4, 0, 0, 1, 6],
         [3, 0, 0, 0, 0, 7],
         [0, 3, 1, 0, 0, 8],
         [0, 0, 6, 7, 8, 0]]
    selected = [0] * V
    no_edge = 0
    selected[0] = True
    print("Edge : Weight\n")
    while no_edge < V - 1:
        minimum = INF
        x = 0
        y = 0
        for i in range(V):
            if selected[i]:
                for j in range(V):
                    if (not selected[j]) and G[i][j]:
                        # not in selected and there is an edge
                        if minimum > G[i][j]:
                            minimum = G[i][j]
                            x = i
                            y = j
        print(f"{x}->{y}: {G[x][y]}")
        selected[y] = True
        no_edge += 1

prim()

# Prim's Algorithm (ładwiejsze przepisanie)

In [ ]:
def prim_v2():
    import networkx as nx

    g = nx.Graph()
    g.add_edge(0, 1, weight=2)
    g.add_edge(0, 2, weight=3)
    g.add_edge(0, 3, weight=3)
    g.add_edge(1, 2, weight=4)
    g.add_edge(1, 4, weight=3)
    g.add_edge(2, 4, weight=1)
    g.add_edge(2, 5, weight=6)
    g.add_edge(3, 5, weight=7)
    g.add_edge(4, 5, weight=8)

    T = nx.minimum_spanning_tree(g, algorithm="prim")
    print("Minimum Spanning Tree", T)
    print("Weight sum: ", sum([T.get_edge_data(u, v).get("weight", 0) for u, v in T.edges()]))
    for u, v in T.edges():
        weight = T.get_edge_data(u, v).get("weight", 0)
        print(f"{u} -- {v} == {weight}")

prim_v2()

# Kolorowanie

In [ ]:
# Python program for solution of M Coloring
# problem using backtracking
def coloring():
    import pandas as pd
    class Graph():

        def __init__(self, vertices):
            self.V = vertices
            self.graph = [[0 for column in range(vertices)]
                          for row in range(vertices)]

            # A utility function to check if the current color assignment

        # is safe for vertex v
        def isSafe(self, v, colour, c):
            for i in range(self.V):
                if self.graph[v][i] == 1 and colour[i] == c:
                    return False
            return True

        # A recursive utility function to solve m
        # coloring  problem
        def graphColourUtil(self, m, colour, v):
            if v == self.V:
                return True

            for c in range(1, m + 1):
                if self.isSafe(v, colour, c) == True:
                    colour[v] = c
                    if self.graphColourUtil(m, colour, v + 1) == True:
                        return True
                    colour[v] = 0

        def graphColouring(self, m):
            colour = [0] * self.V
            if self.graphColourUtil(m, colour, 0) == False:
                return False

            # Print the solution
            print("Solution exist and Following are the assigned colours:")
            df = pd.DataFrame(colour)
            display(df)
            return True


    # Driver Code
    g = Graph(7)
    g.graph = [
        [0, 1, 1, 0, 0, 0, 0],
        [1, 0, 1, 0, 0, 1, 0],
        [1, 1, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 1, 0, 1, 1],
        [0, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 1, 1, 0, 0],
    ]
    m = 3
    g.graphColouring(m)

coloring()
    # This code is contributed by Divyanshu Mehta

Solution exist and Following are the assigned colours:


,0
0,1
1,2
2,3
3,2
4,3
5,1
6,1


# Hopcroft-Karp is an algorithm that takes a bipartite graph as input and produces a maximum-cardinality matching as output

In [ ]:
def hk():
    import networkx as nx
    from networkx.algorithms import bipartite

    g = nx.Graph()
    g.add_nodes_from(["A", "B", "C", "D", "E"], bipartite=0)
    g.add_nodes_from([1, 2, 3, 4, 5], bipartite=1)

    g.add_edge("A", 1)
    g.add_edge("A", 2)
    g.add_edge("A", 3)
    g.add_edge("B", 1)
    g.add_edge("C", 2)
    g.add_edge("C", 3)
    g.add_edge("C", 4)
    g.add_edge("D", 3)
    g.add_edge("D", 5)
    g.add_edge("E", 3)
    g.add_edge("E", 4)

    display(bipartite.hopcroft_karp_matching(g))


hk()

{'E': 3,
 'B': 1,
 'D': 5,
 'A': 2,
 'C': 4,
 1: 'B',
 2: 'A',
 3: 'E',
 4: 'C',
 5: 'D'}